In [4]:
import handcalcs.render
import forallpeople
from handcalcs.decorator import handcalc
import pandas as pd
forallpeople.environment("structural", top_level=True)

In [5]:
W1 =1430 *kN 
W2 = 1300  *kN 
W3 = 1000 * kN

H_n = [4,3,3][::-1]

H_n_ = [j * m for j in H_n]

W_list = [W3,W2,W1]
index =  [f"Floor {i + 1}" if i != len(H_n_) else "Roof" for i,j in enumerate(H_n_,1)][::-1]

WH = pd.DataFrame([W_list,H_n_])
WH = WH.T.set_index([index])
WH[3] =   WH.loc[::-1, 1].cumsum()[::-1]
WH[4] =   WH[0]*WH[3]
WH.columns = ["W","H","H_n","WxH"]
WH

,W,H,H_n,WxH
Roof,1.000 MN,3.000 m,10.000 m,10.000 MN·m
Floor 3,1.300 MN,3.000 m,7.000 m,9.100 MN·m
Floor 2,1.430 MN,4.000 m,4.000 m,5.720 MN·m


## Parameters

In [6]:

zone = 0.4 
SeismicSource_Type = "A"
Soil_Profile = "SD"
I = 1
R = 8.5
Distance = 5
C_t = 0.731
mode = "long"
precision = 5
@handcalc(override=mode, jupyter_display=True, precision=precision)
def C_t_finder():
    

@handcalc(override="params", jupyter_display=True, precision=precision)
def parameters(zone,SeismicSource_Type,Soil_Profile,I,R,Distance,H_n,C_t):
    Zone = zone
    SeismicSource = SeismicSource_Type
    Soil = Soil_Profile
    I = I
    R = R
    Distance = Distance * 10 ** 3 * m 
    H_n = sum(H_n)
    C_t = C_t
    
    
@handcalc(override=mode, jupyter_display=True, precision=precision)
def telastic(H_n):
    T = 0.0731 * (sum(H_n) ** (3 / 4)) # Equation 208-12
    return T

@handcalc(override=mode, jupyter_display=True, precision=precision)
def ircd(i):
    if i == "I":x = 1.5
    elif i == "II":x = 1.25
    elif i == "III":x = 1
    elif i == "IV":x = 1
    elif i == "V":x = 1
    return x


@handcalc(override=mode, jupyter_display=True, precision=precision)
def narcd(Distance):
    if Distance >= 5 and Distance <= 10:Na = 1.2 - (((Distance - 5) / 5) * (1.2 - 1)) # Table 208-6
    elif Distance >= 2 and Distance <= 5:Na = 1.5 - (((Distance - 2) / 3) * (1.5 - 1.2)) # Table 208-6
    return Na


@handcalc(override=mode, jupyter_display=True, precision=precision)
def nvrcd(Distance):

    if Distance >= 5 and Distance <= 10:Nv = 1.6 - (((Distance - 5) / 5) * (1.6 - 1.2)) # Table 208-5
    elif Distance >= 2 and Distance <= 5:Nv = 2 - (((Distance - 2) / 3) * (2 - 1.6)) # Table 208-5
    elif Distance >= 10 and Distance <= 15:Nv = 1.2 - (((Distance - 10) / 5) * (1.2 - 1)) # Table 208-5
    return Nv


@handcalc(override=mode, jupyter_display=True, precision=precision)
def carcd(na, Soil_Profile):
    if Soil_Profile == "SA":Ca = 0.32 * na # Table 208-7
    elif Soil_Profile == "SB":Ca = 0.4 * na # Table 208-7
    elif Soil_Profile == "SC":Ca = 0.4 * na # Table 208-7
    elif Soil_Profile == "SD":Ca = 0.44 * na # Table 208-7
    elif Soil_Profile == "SE":Ca = 0.44 * na # Table 208-7
    return Ca


@handcalc(override=mode, jupyter_display=True, precision=precision)
def cvrcd(nv, Soil_Profile):
    if Soil_Profile == "SA":Cv = 0.32 * nv # Table 208-8
    elif Soil_Profile == "SB":Cv = 0.4 * nv # Table 208-8 
    elif Soil_Profile == "SC":Cv = 0.56 * nv# Table 208-8
    elif Soil_Profile == "SD":Cv = 0.64 * nv # Table 208-8
    elif Soil_Profile == "SE":Cv = 0.96 * nv # Table 208-8
    return Cv


@handcalc(override=mode, jupyter_display=True, precision=precision)
def BaseShear(cv, I, r, t, w):
    V = ((cv * I) / (r * t)) * w # Equation 208-8
    return V


@handcalc(override=mode, jupyter_display=True, precision=precision)
def MaxShear(ca, I, r, w):
    V = ((2.5 * ca * I) / r) * w # Equation 208-9
    return V


@handcalc(override=mode, jupyter_display=True, precision=precision)
def MinShear(ca, I, w):
    V = 0.11 * ca * I * w # Equation 208-10
    return V


@handcalc(override=mode, jupyter_display=True, precision=precision)
def Zone4(z, nv, I, r, w):
    V = ((0.8 * z * nv * I) / r) * w # Equation 208-11
    return V

# @handcalc(override=mode, jupyter_display=True, precision=2)
# def FBaseShear(v, vmax, V_min, V_zone4):

#     if V_min > V_zone4:V_min = V_min
#     else:V_min = V_zone4
#     if v < V_min:BaseShear = V_min
#     elif v > vmax:BaseShear = vmax
#     else:BaseShear = v
#     return BaseShear

@handcalc(override=mode, jupyter_display=True, precision=precision)
def FBaseShear(v, vmax, V_min, V_zone4):

    if v < max(V_min,V_zone4):BaseShear = max(V_min,V_zone4)
    elif v > vmax:BaseShear = vmax
    else:BaseShear = v
    return BaseShear

@handcalc(override=mode, jupyter_display=True, precision=precision)
def ft(t, v):
    if t > 0.7 : ft = 0.07 * t * v # Equation 208-16
    elif t < 0.7 : ft = 0
    if ft > (0.25 * v):ft = 0.25 * v # Equation 208-16
    return ft


@handcalc(override=mode, jupyter_display=True, precision=2)
def lateral_force(v, ft, WixHi,WixHi_Total,floor):
    F = ((v - ft) * WixHi)/WixHi_Total
    return F


@handcalc(override=mode, jupyter_display=True, precision=precision)
def lateral_force_top(v, ft, coefficient,WxH_Total,T):
    F = (((v- ft) * coefficient)/WxH_Total) + ft
    return F




# print(type(Distance))
sadasd = parameters(zone, SeismicSource_Type, Soil_Profile, I, R, Distance, H_n,C_t)








IndentationError: expected an indented block (1569996420.py, line 14)

## Total Weight

In [ ]:
%%render 
W = sum((W1,W2,W3))

<IPython.core.display.Latex object>

## Nv,Na,Cv,Ca

In [ ]:

Na = narcd(Distance) # Table 208-5
Nv = nvrcd(Distance) # Table 208-6
Ca = carcd(Na, Soil_Profile)# Table 208-7
Cv = cvrcd(Nv, Soil_Profile)# Table 208-8
T = telastic(H_n) # Equation 208-12

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Base Shear

In [ ]:

BaseShear = BaseShear(Cv, I, R, T, W) # Equation 208-8
MaxShear = MaxShear(Ca, I, R, W)# Equation 208-9
MinShear = MinShear(Ca, I, W)# Equation 208-10
Zone4 = Zone4(zone, Nv, I, R, W)# Equation 208-11
FBaseShear = FBaseShear(BaseShear, MaxShear, MinShear, Zone4)
ft = ft(T, FBaseShear)# Equation 208-16

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## F1,F2 AND F3

In [ ]:
WxH_Total = WH["WxH"].sum()
asdas = [lateral_force(FBaseShear, ft, j,WxH_Total,i) for i,j in enumerate(WH["WxH"][::-1],1) ]
LateralPd = pd.DataFrame(asdas,index = [f"F{i}" for i,j in enumerate(asdas,1)],columns = ["Forces"])
LateralPd

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

,Forces
F1,133.493 kN
F2,212.375 kN
F3,233.379 kN


In [ ]:
import numpy as np
from math import sqrt
D_e = np.array([6,6])
H_n_ = H_n[0]

In [ ]:
%%render

A_e = 0.3 * 6
A_c = sum(A_e*(0.2+(D_e/H_n_))**2)
C_t = 0.0743*sqrt(A_c)

<IPython.core.display.Latex object>

In [10]:
%%render
e_x = abs(22.678 - 22.282)
e_y = abs(22.336 - 22.951)
e = sqrt(e_x**2 + e_y**2)
e_natural = 0.05 * 44
if e < e_natural:e_ = e_natural
    

<IPython.core.display.Latex object>

In [ ]:
from math import sqrt